<a href="https://colab.research.google.com/github/Ruheena-S/Hierarchical-classification-Loss-Functions-in-Image-Classification/blob/main/ResNet50_CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch_optimizer torchmetrics
!nvidia-smi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 42.2 MB/s eta 0:00:00
Mon Mar 13 21:54:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    34W /  70W |   5449MiB / 15360MiB |      0%    

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch_optimizer import Ranger
from torchvision.datasets import CIFAR100
from torch.utils.data import DataLoader
from torch.utils.data import random_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
torch.manual_seed(43)

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Define transforms for the dataset
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [ ]:
# Load the dataset

dataset = CIFAR100(root='./data', train=True, download=True, transform=transform_train)

test_dataset = CIFAR100(root='./data', train=False, download=True, transform=transform_test)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
val_size = 5000
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])


trainloader = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
valloader = DataLoader(val_ds, batch_size=100, num_workers=2, pin_memory=True)
testloader = DataLoader(test_dataset, batch_size =100, num_workers=4, pin_memory=True)

In [ ]:
# Define the ResNet50 model and freeze layers

model = torchvision.models.resnet50(pretrained=True)
model.conv1 = nn.Conv2d(3, 64, kernel_size = (3,3), padding = (1, 1), bias = False)
model.maxpool = nn.Identity()

for param in model.parameters():
    param.requires_grad = True
for param in model.layer4.parameters():
    param.requires_grad = True

model.fc = nn.Linear(2048, 100)

model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentu

In [ ]:
# Define the loss function and optimizer

criterion = nn.CrossEntropyLoss().to(device)
#optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

# optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
optimizer = Ranger(model.parameters(), lr=0.001, weight_decay=0.005) 
#optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Train the model

BEST_VAL_METRIC = 0
BEST_MODEL = None


for epoch in range(20):  # number of epochs
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print('Epoch: %d Train Loss: %.3f Train Acc: %.3f' % (epoch+1, train_loss/(i+1), 100.*correct/total))

    # Validate the model
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(valloader, 0):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print('Val Loss: %.3f Val Acc: %.3f' % (val_loss/(i+1), 100.*correct/total))

    if (100.*correct/total) > BEST_VAL_METRIC:
        BEST_VAL_METRIC = (100.*correct/total)
        BEST_MODEL = model.state_dict() 
        #torch.save(model, "/content/drive/MyDrive/MTP_Phase2/saved_models/resnet50_cifar100_OPT.pth")
        torch.save(model, "/content/drive/MyDrive/Ruheena/MTP/saved_models/resnet50_cifar100_RangerOPT.pth")


/usr/local/lib/python3.9/dist-packages/pytorch_ranger/ranger.py:172: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1420.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


Epoch: 1 Train Loss: 2.762 Train Acc: 33.502
Val Loss: 1.616 Val Acc: 54.880
Epoch: 2 Train Loss: 1.278 Train Acc: 63.173
Val Loss: 1.283 Val Acc: 62.680
Epoch: 3 Train Loss: 1.007 Train Acc: 70.273
Val Loss: 1.095 Val Acc: 68.300
Epoch: 4 Train Loss: 0.861 Train Acc: 74.053
Val Loss: 1.207 Val Acc: 66.360
Epoch: 5 Train Loss: 0.782 Train Acc: 76.593
Val Loss: 1.109 Val Acc: 69.020
Epoch: 6 Train Loss: 0.709 Train Acc: 78.447
Val Loss: 1.014 Val Acc: 71.960
Epoch: 7 Train Loss: 0.651 Train Acc: 80.231
Val Loss: 1.167 Val Acc: 68.280
Epoch: 8 Train Loss: 0.594 Train Acc: 81.687
Val Loss: 1.051 Val Acc: 71.120
Epoch: 9 Train Loss: 0.555 Train Acc: 82.727
Val Loss: 0.985 Val Acc: 72.340
Epoch: 10 Train Loss: 0.510 Train Acc: 84.036
Val Loss: 1.084 Val Acc: 71.100
Epoch: 11 Train Loss: 0.479 Train Acc: 84.882
Val Loss: 1.027 Val Acc: 72.520
Epoch: 12 Train Loss: 0.432 Train Acc: 86.431
Val Loss: 0.974 Val Acc: 73.440
Epoch: 13 Train Loss: 0.415 Train Acc: 86.791
Val Loss: 1.082 Val Acc: 71

In [ ]:
# Test the model

model.load_state_dict(BEST_MODEL)

model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for i, (inputs, labels) in enumerate(testloader, 0):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print('Test Loss: %.3f Test Acc: %.3f' % (test_loss/(i+1), 100.*correct/total))


Test Loss: 1.165 Test Acc: 74.390
